In [1]:
from src.models import Sequential
from src.layers import *
from src.activation_functions import *
from src.loss_functions import *

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
the_image = np.random.random((1, 1, 28, 28))

In [3]:
conv = Conv2D(1, 3, 1, 1)

In [4]:
conv.forward(the_image).shape

(1, 1, 28, 28)

In [5]:
the_error = np.load('error.npy')

In [6]:
new_error = np.expand_dims(the_error[0], 0)
new_error.shape

(1, 1, 28, 28)

In [7]:
weig = conv.weights

In [8]:
weig.shape

(1, 3, 3)

In [9]:
new_error_padded = np.pad(new_error, ((0, 0), (0, 0), (1, 1), (1, 1)))
new_error_padded.shape

(1, 1, 30, 30)

In [10]:
conv.backward(new_error, 1)

array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,
      

In [11]:
conv.weights

array([[[-0.00544742,  0.24769348,  0.26802978],
        [-0.347622  ,  0.07393277,  0.17415562],
        [ 0.7301308 , -0.05243952,  0.43054223]]], dtype=float32)

In [12]:
test_error = np.asarray([[
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]])
print(test_error.shape)

test_error_padded = np.pad(test_error[0], 1)
test_error2 = np.expand_dims(test_error_padded, (0, 1))

(1, 6, 5)


In [13]:
test_error.shape

(1, 6, 5)

In [14]:
test_error

array([[[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]]])

In [15]:
weig = np.asarray([[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
],
[
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
]])

In [16]:
weig.shape

(2, 3, 3)

In [36]:
error_conv = np.zeros((1, 1, 28, 28))
error_conv.shape

(1, 1, 28, 28)

In [37]:
for b in range(error_conv.shape[0]):
    for f in range(error_conv.shape[1]):
        for h in range(error_conv.shape[2]):
            for w in range(error_conv.shape[3]):
                error_conv[b, f, h, w] = np.sum(
                    new_error_padded[b, f, h:h+3, w:w+3] * weig[f]
                )

In [32]:
error_conv.shape

(1, 1, 30, 30)

In [33]:
error_conv[0]

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.

In [21]:
import scipy

In [22]:
scipy.signal.correlate2d(test_error[0][0], weig[0])

ValueError: correlate2d inputs must both be 2-D arrays

In [ ]:
test_error[0][0][1:-1, 1:-1] * error_conv[0][0]

In [64]:
test_error = np.asarray([[
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]])

test_error = test_error[0, 1:-1, 1:-1]

test_inp = np.asarray([[
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [2, 2, 2, 2, 3],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4],
], [
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [2, 2, 2, 2, 3],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4],
]])

test_weig = np.asarray([[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
],
[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
]])

In [65]:
test_grad = np.zeros((2, 3, 3))

In [70]:
for b in range(test_grad.shape[0]):
    for h in range(test_grad.shape[1]):
        for w in range(test_grad.shape[2]):
            test_grad[b, h, w] = np.sum(
                test_inp[b, h:h+test_error.shape[0], w:w+test_error.shape[1]] * test_error
            )

In [71]:
print(test_grad)

[[[15. 18. 25.]
  [21. 23. 28.]
  [30. 31. 34.]]

 [[15. 18. 25.]
  [21. 23. 28.]
  [30. 31. 34.]]]


In [69]:
test_grad * 0.01

array([[[0.15, 0.18, 0.25],
        [0.21, 0.23, 0.28],
        [0.3 , 0.31, 0.34]],

       [[0.15, 0.18, 0.25],
        [0.21, 0.23, 0.28],
        [0.3 , 0.31, 0.34]]])